In [104]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility as iv
from math import exp, sqrt, log, fabs
import matplotlib.pyplot as plt
from itertools import repeat
import os

In [105]:
work_dir = os.getcwd()
work_dir

'C:\\Users\\User\\Documents\\GitHub\\OptionStrategy'

In [106]:
def df_generate(product, date):
    # read the csv data
    csvName = product + "_" + date + '.csv'
    Path_default_readcsv = os.path.join(work_dir, 'Data', csvName)
    df = pd.read_csv(Path_default_readcsv, encoding = 'cp950')
    
    return df

In [107]:
def Flag_Moneyness(df, S, code):
    # append a column of ITM/OTM toward the put call dataframe
    if code == 'call':
        df["Moneyness"] = ["OTM" if k > S else "ITM" for k in df["履約價"]]
    else:
        df["Moneyness"] = ["OTM" if k < S else "ITM" for k in df["履約價"]]
    return df

In [108]:
def todayStockPrice(date):
    product = 'MTX'
    csvName = product + "_" + date + '.csv'
    Path_default_readcsv = os.path.join(work_dir, 'Data', csvName)
    df = pd.read_csv(Path_default_readcsv, encoding = 'cp950')
    S = float(df["最後成交價"][0])
    return S

In [109]:
def IV_List(df, Stock, d, flag):
    # Write the Implied Volatility to the dataframe
    
    start_date = date(2020, int(d[1]), int(d[2:]))
    end_date = date(2020, 2, 19) # the delivery date in Feb. 
    
    S = float(Stock)
    r = 0
    q = 0
    t = np.busday_count(start_date, end_date)/252    
    
    list_IV = []
    
    #record the NA IV
    volumeNA = 0
    boundaryNA = 0
    NBBONA = 0
    for i in range(len(df)):
        if df["*一般交易時段成交量"][i] == 0: # elimanate the option with volume = 0
            list_IV.append("-")
            volumeNA += 1
        elif df["最後最佳買價"][i] == "-" or df["最後最佳賣價"][i] == "-":
            list_IV.append("-")
            NBBONA += 1
        else:
            K = float(df["履約價"][i])
            # two cases
            if flag == "Call":
                call_price = (float(df["最後最佳買價"][i]) + float(df["最後最佳賣價"][i]))/2
                intrinsic_c = fabs(max(S - K, 0.0))
                if call_price < intrinsic_c or call_price >= S: #eliminate the violation of option pricing
                    list_IV.append("-")
                    boundaryNA += 1
                else:
                    call_iv = iv(price = call_price, 
                             flag = 'c', 
                             S = Stock, 
                             K = K, 
                             t = t, 
                             r = r,
                             q = q)
                    list_IV.append(call_iv)
                    
            elif flag == "Put":
                put_price = (float(df["最後最佳買價"][i]) + float(df["最後最佳賣價"][i]))/2
                intrinsic_p = fabs(max(K - S, 0.0))
                if put_price < intrinsic_p or put_price >= K: #eliminate the violation of option pricing
                    list_IV.append("-")
                    boundaryNA += 1
                else:
                    put_iv = iv(price = put_price, 
                             flag = 'p', 
                             S = S, 
                             K = K, 
                             t = t, 
                             r = r,
                             q = q)
                    list_IV.append(put_iv)
                    
    categoryNA = [volumeNA, boundaryNA, NBBONA]
    return list_IV, categoryNA

In [116]:
if __name__ ==  '__main__':
    date_ = '0130'
    S = todayStockPrice(date = date_ )
    
    df = df_generate(product = 'TXO', date = date_ )
    df_put = df[df.買賣權 == 'Put'].reset_index(drop=True)
    df_call = df[df.買賣權 == 'Call'].reset_index(drop=True)
    
    df_put = Flag_Moneyness(df_put, S = S, code = 'Put')
    list_IV, categoryNAPut = IV_List(df = df_put, Stock = S, d = date_ , flag = 'Put')
    df_put["IV"] = list_IV
    
    df_call = Flag_Moneyness(df_call, S = S, code = 'Call')
    list_IV, categoryNACall = IV_List(df = df_call, Stock = S, d = date_ , flag = 'Call')
    df_call["IV"] = list_IV